In [1]:
import sys

In [2]:
sys.version

'3.6.9 (default, Apr 20 2021, 10:09:27) \n[GCC 9.3.0]'

In [3]:
%matplotlib inline
import os
from pprint import pprint
import matplotlib.pyplot as plt
import sciunit
import sciunit.scores
import neuronunit
from neuronunit.models.reduced import ReducedModel
from neuronunit.capabilities import ProducesSpikes

In [4]:
# Assumes imported neuronunit is from source, e.g. pip install -e
path = os.path.join(neuronunit.__path__[0],'models/NeuroML2/LEMS_2007One.xml')
# Instantiate three identical models
models = [ReducedModel(path, name='Izhikevich_{}'.format(i), backend='jNeuroML') for i in range(4)]

In [5]:
# Change the parameters of the second model
models[1].set_attrs(**{'izhikevich2007Cell':{'a':'0.04 per_ms'}})
models[1].name = 'Izhikevich_alt'

In [ ]:
for model in models:
    plt.plot(model.get_membrane_potential(),label=model)
plt.legend();

In [ ]:
# A new M2M test which will compare the equality of spike counts across models
from scidash_api.tests import RandomTest

In [ ]:
test = RandomTest(observation={'mean':8, 'std':3 })

In [ ]:
s = test.judge(models)

In [ ]:
s

In [ ]:
x = test.json(string=False, add_props=True)
pprint(x)

In [ ]:
def check_url(x):
    if not x.json(string=False).get('url'):
        print("Model has no associated url; please set the url attribute of the model")
check_url(models[0])

In [ ]:
# How do you want your GitHub URLs?  
lems_url = ("https://github.com/scidash/neuronunit/blob/master/neuronunit/"
            "models/NeuroML2/LEMS_2007One.xml")
models[0]._url = lems_url
models[1]._url = lems_url
models[2]._url = lems_url
models[3]._url = lems_url

In [ ]:
check_url(models[0])

In [ ]:
# Has fewer attributes because most of them came from the LEMS file so those are assumed
# to still be the values in memory.  If we need to extract these then I need to find a
# way to read them from the file; however, there may be no general way to do so for all
# LEMS files, and certainly not for all possible models.  
x = models[0].json(string=False)
pprint(x)

In [ ]:
# Has more attributes because some of the original attributes from the LEMS file 
# were replaced with new ones in this session
x = models[1].json(string=False)
pprint(x)

In [ ]:
test2 = RandomTest(observation={'mean':3, 'std':8},name='Another Test')
test3 = RandomTest(observation={'mean':2, 'std':8},name='Another Test 2')
test4 = RandomTest(observation={'mean':4, 'std':8},name='Another Test 3')
test5 = RandomTest(observation={'mean':1, 'std':8},name='Another Test 4')
test1 = RandomTest(observation={'mean':5, 'std':4},name='Another Test 5')
test8 = RandomTest(observation={'mean':4, 'std':8},name='Another Test 3')
test9 = RandomTest(observation={'mean':1, 'std':8},name='Another Test 4')
test10 = RandomTest(observation={'mean':5, 'std':4},name='Another Test 5')
suite = sciunit.TestSuite([test,test2,test3,test4,test5,test8],name='My Test Suite')
x = suite.json(add_props=True,string=False)
pprint(x)

In [ ]:
score_matrix = suite.judge(models)

In [ ]:
from scidash_api import client

In [ ]:
import json,urllib.request
data = urllib.request.urlopen('https://raw.githubusercontent.com/MetaCell/scidash-api/development/config.json').read()
config = json.loads(data)
ip = config["prod"]["ip"]
print (ip)

In [ ]:
client_instance = client.ScidashClient({"base_url": ip})

client_instance.login(username='admin', password='admin_test')

print(client_instance.token)

In [ ]:
responses = client_instance.upload_suite_score(suite, score_matrix)

In [ ]:
for response in responses:
    pprint(response.json())